In [1]:
%load_ext autoreload
%autoreload 2

from lrp import *
from lrp.data import Data
from lrp import read_mnist
import os
from lrp.evaluate_rule import HeatmapEval
from lrp import utils
import os

# train a good cnn and a good fcn
mnist = read_mnist.read_data_sets("{}/datasets/mnist".format(os.environ["TF_PROJECTS"]), one_hot=True)
data = Data(X=mnist.train.images, y=mnist.train.labels,
                X_val=mnist.validation.images, y_val=mnist.validation.labels,
                X_test=mnist.test.images, y_test=mnist.test.labels)
fcn = Network([Format(), FirstLinear(784), ReLU(), NextLinear(10)], data.X, data.y_)
cnn = Network([Format(), FirstConvolution([5, 5, 1, 32]), ReLU(), Pooling(),
               NextConvolution([5, 5, 32, 64]), ReLU(), Pooling(),
               NextLinear(1024), ReLU(),
               NextLinear(10)],
               data.X, data.y_)


sess = tf.Session()
    
fcn.set_session(sess)
cnn.set_session(sess)

sess.run(tf.global_variables_initializer())

try:
    fcn.load_params("trained_models/example_fcn")
except Exception as e:
    print(type(e), e)
    print("Train new fcn")
    fcn.fit(data, lambda i, val_acc: max([0]+val_acc)<=0.98)
    fcn.save_params("trained_models/example_fcn")

try:
    cnn.load_params("trained_models/example_cnn")
except Exception as e:
    print(type(e), e)
    print("Train new cnn")
    cnn.fit(data, lambda i, val_acc: max([0]+val_acc)<=0.98)
    cnn.save_params("trained_models/example_cnn")   
    

samples = {data.X: data.X_test[:16], data.y_: data.y_test[:16]}
y_ = data.y_test[:16]

/home/niels/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please write your own downloading logic.
Extracting /home/niels/Dokumente/Informatik/bachelor/tensorflow/datasets/mnist/train-images-idx3-ubyte.gz
Extracting /home/niels/Dokumente/Informatik/bachelor/tensorflow/datasets/mnist/train-labels-idx1-ubyte.gz
Extracting /home/niels/Dokumente/Informatik/bachelor/tensorflow/datasets/mnist/t10k-images-idx3-ubyte.gz
Extracting /home/niels/Dokumente/Informatik/bachelor/tensorflow/datasets/mnist/t10k-labels-idx1-ubyte.gz
Network layer mappings: (?, 784)
-> (?, 784)
-> (?, 784)
-> (?, 10)
Network layer mappings: (?, 784)
-> (?, 24, 24, 32)
-> (?, 24, 24, 32)
-> (?, 12, 12, 32)
-> (?, 8, 8, 64)
-> (?, 8, 8, 64)
-> (?, 4, 4, 64)
-> (?, 1024)
-> (?, 1024)
-> (?, 10)
INFO:tensorflow:Restoring parameters from trained_models/example_fcn/model.ckpt
INFO:tensorflow:Restoring parameters from trained_models/example_cnn/model.ckpt


In [4]:
nns = [fcn, cnn]


rules = ["simple",
         ("simple", {"reference": data.X_val[:100]}),
         "deeptaylor",
         ("deeptaylor", {"reference": data.X_val[:100]}),
         "zbab",
         ("zbab", {"reference": data.X_val[:100]}),
        "ab",
        ("ab", {"reference": data.X_val[:100]})]

evaluator = HeatmapEval(nns, ["fcn", "cnn"],
                        rules, ["simple", "simple-ref", "deeptaylor", "deeptaylor-ref", "zbab", "zbab-ref", "ab", "ab-ref"],
                        data)



In [5]:
os.makedirs("evaluate/heatmaps", exist_ok=True)

sample_X = data.X_test[1:16][::2] #[1:16][::2]
sample_y = data.y_test[1:16][::2] #[1:16][::2]
print("sample_X", sample_X.shape)

utils.visualize(2*sample_X - 1, utils.graymap, "evaluate/heatmaps/original")

for nn_name, nn in zip(["fcn", "cnn"], nns):

    for rule_name, H in zip(evaluator.rule_names, evaluator.heatmaps[nn]):
        print("Evaluate", rule_name, "on", nn_name)
        heatmaps, prediction, refs = sess.run([H, nn.y, nn.refs], feed_dict={data.X: sample_X, data.y_: sample_y})
        utils.visualize(heatmaps, utils.heatmap, "evaluate/heatmaps/{}-{}".format(
            nn_name, rule_name))
        print("Visualized heatmaps")

print("--------- COMPARE -----------------")
#evaluator.compare()
#evaluator.plot_effect("evaluate")
#plt.show()

sample_X (8, 784)
len 8
x.shape (8, 784)
Evaluate simple on fcn
len 8
x.shape (8, 784)
Visualized heatmaps
Evaluate simple-ref on fcn
len 8
x.shape (8, 784)
Visualized heatmaps
Evaluate deeptaylor on fcn
len 8
x.shape (8, 784)
Visualized heatmaps
Evaluate deeptaylor-ref on fcn
len 8
x.shape (8, 784)
Visualized heatmaps
Evaluate zbab on fcn
len 8
x.shape (8, 784)
Visualized heatmaps
Evaluate zbab-ref on fcn
len 8
x.shape (8, 784)
Visualized heatmaps
Evaluate ab on fcn
len 8
x.shape (8, 784)
Visualized heatmaps
Evaluate ab-ref on fcn
len 8
x.shape (8, 784)
Visualized heatmaps
Evaluate simple on cnn
len 8
x.shape (8, 784)
Visualized heatmaps
Evaluate simple-ref on cnn
len 8
x.shape (8, 784)
Visualized heatmaps
Evaluate deeptaylor on cnn
len 8
x.shape (8, 784)
Visualized heatmaps
Evaluate deeptaylor-ref on cnn
len 8
x.shape (8, 784)
Visualized heatmaps
Evaluate zbab on cnn
len 8
x.shape (8, 784)
Visualized heatmaps
Evaluate zbab-ref on cnn
len 8
x.shape (8, 784)
Visualized heatmaps
Evaluat

In [17]:
# Since the original and the ref rules look exactly the same, let's check if they are

class_filter = 2*data.y_ - np.ones(10) # np.eye(10)[3]

for rule in ["deeptaylor", "zbab", "simple", "ab"]:


    def check_conservation(h, y, msg):
        print(msg, "error: ", np.sum(h, axis=tuple(range(1, len(h.shape))))/y)

    Href_cnn = cnn.lrp(class_filter, rule, reference=data.X_val[:100])
    H_cnn= cnn.lrp(class_filter, rule)

    h, href, y = sess.run([H_cnn, Href_cnn, cnn.y], feed_dict=samples)
    y = np.sum(y*data.y_test[:16], axis=1)

    check_conservation(href, y, "cnn-{}-ref".format(rule))
    check_conservation(h, y, "cnn-{}".format(rule))

    utils.visualize(h, utils.heatmap, "test/cnn-{}.png".format(rule))
    utils.visualize(href, utils.heatmap, "test/cnn-{}-ref.png".format(rule))


    Href_fcn = fcn.lrp(class_filter, rule, reference=data.X_val)
    H_fcn= fcn.lrp(data.y_, rule)

    h, href, y = sess.run([H_fcn, Href_fcn, fcn.y], feed_dict=samples)
    y = np.sum(y*data.y_test[:16], axis=1)

    check_conservation(href, y, "fcn-{}-ref".format(rule))
    check_conservation(h, y, "fcn-{}".format(rule))

    utils.visualize(h, utils.heatmap, "test/fcn-{}.png".format(rule))
    utils.visualize(href, utils.heatmap, "test/fcn-{}-ref.png".format(rule))


cnn-deeptaylor-ref error:  [11.49662065 10.57243997 18.94011405 11.27546667 11.51927742 13.49278846
 14.27219906 13.22741171 25.75523549 13.39925684 12.28481916 11.8147145
 14.43137113 12.36739853 15.03508954 16.80007595]
cnn-deeptaylor error:  [11.49662065 10.57243881 18.94011574 11.27546815 11.51927665 13.49278946
 14.27219906 13.22741171 25.75523549 13.3992577  12.28482052 11.8147145
 14.43137017 12.36739853 15.03508729 16.80007767]
fcn-deeptaylor-ref error:  [  15.22099157   16.62481706   14.64105579   15.64407595   22.90346707
   14.84643365   14.93316894   15.77011593 -113.1864897    26.89411367
   12.96547047   22.35950968   18.99958796   12.96472452   13.09914546
   17.19113669]
fcn-deeptaylor error:  [1.         1.00000008 1.00000008 0.99999991 1.00000009 0.99999987
 1.00000006 0.99999992 1.         1.         0.99999994 0.99999991
 1.         0.99999985 0.9999999  1.        ]
cnn-zbab-ref error:  [11.49662182 10.57243997 18.94011405 11.27546667 11.51927742 13.49278846
 14.272

In [5]:
layer = 0

print("direct")
layer = 0
H= cnn.lrp(data.y_,"deeptaylor", reference=data.X_val)
h, y = sess.run([H, cnn.y], feed_dict=samples)
print("total",
          np.sum(h, axis=tuple(range(1, len(h.shape))))/np.sum(y*y_, axis=1))
utils.visualize(h, utils.heatmap_original, "test/cnn-deeptaylor-ref.png")



direct
total [1.00000007 1.         0.99999989 1.         0.9999999  1.00000006
 0.99999993 0.99999993 0.9999999  1.         0.99999992 1.
 1.         1.00000009 1.         1.00000011]


In [3]:
H, C = fcn.layerwise_lrp(data.y_,"deeptaylor", reference=data.X_val)
h, c, y = sess.run([H, C, fcn.y], feed_dict=samples)
print("deeptaylor ref layerwise", c)
print("total",
          np.sum(h[1], axis=tuple(range(1, len(h[0].shape))))/np.sum(y*y_, axis=1))


deeptaylor ref layerwise [1.0000005, 1.0, 1.0, 1.0]
total [1.         0.99999993 1.00000006 1.         1.         1.00000011
 1.00000019 1.         1.00000028 0.99999993 0.99999989 0.99999978
 1.00000008 1.0000002  1.00000007 0.99999985]


In [12]:
# Since the original and the ref rules look exactly the same, let's check if they are

samples = {data.X: data.X_test[:16], data.y_: data.y_test[:16]}

#H = cnn.lrp(data.y_, "deeptaylor")
Href = fcn.lrp(data.y_, "simple", reference=data.X_val)

href = sess.run(Href, feed_dict=samples)

utils.visualize(href, utils.heatmap_original, "test/fcn-simple-ref.png")

p = np.sum(href, axis=1)
y = sess.run(fcn.y, feed_dict=samples)*data.y_test[:16]
y = np.sum(y, axis=1)
error = p/y
print(error)


[1.00000021 1.00000022 1.00000024 1.00000079 1.00000207 1.00000011
 0.99999991 0.99999918 1.00000197 1.00000007 1.00000022 1.
 0.99999975 0.9999999  0.99999972 0.99999963]
